### Modelo.py

In [29]:
from pomegranate import *

# Nodo Lluvia, no tiene padres
lluvia = Node(DiscreteDistribution({
    "ninguna": 0.7,
    "suave": 0.2,
    "fuerte": 0.1
}), name="lluvia")

# Nodo de Mantenimiento esta condicionado por la lluvia
mantenimiento = Node(ConditionalProbabilityTable([
    ["ninguna","si",0.4],
    ["ninguna","no",0.6],
    ["suave","si",0.2],
    ["suave","no",0.8],
    ["fuerte","si",0.1],
    ["fuerte","no",0.9]
],[lluvia.distribution]), name="mantenimiento")

# nodo Bus esta condicionado por la lluvia y el mantenimiento
bus = Node(ConditionalProbabilityTable([
    ["ninguna","si","a tiempo", 0.8],
    ["ninguna","si","retrasado", 0.2],
    ["ninguna","no","a tiempo", 0.9],
    ["ninguna","no","retrasado", 0.1],
    ["suave","si","a tiempo", 0.6],
    ["suave","si","retrasado", 0.4],
    ["suave","no","a tiempo", 0.7],
    ["suave","no","retrasado", 0.3],
    ["fuerte","si","a tiempo", 0.4],
    ["fuerte","si","retrasado", 0.6],
    ["fuerte","no","a tiempo", 0.5],
    ["fuerte","no","retrasado", 0.5]
],[lluvia.distribution, mantenimiento.distribution]), name="bus")

# Nodo Cita esta condicionada por el Bus
cita = Node(ConditionalProbabilityTable([
    ["a tiempo", "atendida", 0.9],
    ["a tiempo", "perdida", 0.1],
    ["retrasado", "atendida", 0.6],
    ["retrasado", "perdida", 0.4]
],[bus.distribution]), name="cita")

# Creamos una Red Bayesiana y añadimos estados
modelo = BayesianNetwork()
modelo.add_states(lluvia, mantenimiento, bus, cita)

# Añadimos bordes que conecten nodos
modelo.add_edge(lluvia, mantenimiento)
modelo.add_edge(mantenimiento, bus)
modelo.add_edge(lluvia, bus)
modelo.add_edge(bus, cita)

#Modelo Final
modelo.bake()

### Probabilidad.py

In [37]:
# Calculemos la probabilidad para una observacion dada
probabilidad1 = modelo.probability([["ninguna", "no", "a tiempo", "atendida"]])
probabilidad2 = modelo.probability([["fuerte", "si", "a tiempo", "atendida"]])
probabilidad3 = modelo.probability([["suave", "no", "a tiempo", "atendida"]])

print("Probabilidad #1:",probabilidad1)
print("Probabilidad #2:",probabilidad2)
print("Probabilidad #3:",probabilidad3)

Probabilidad #1: 0.34019999999999995
Probabilidad #2: 0.003600000000000003
Probabilidad #3: 0.10079999999999999


### Inferencia.py

In [33]:
# Calculemos las predicciones
predicciones = modelo.predict_proba({
    'bus':'a tiempo'
})

# Visualizemos las predicciones para cada nodo
for nodo, prediccion in zip(modelo.states, predicciones):
    if isinstance(prediccion, str):
        print(f"{nodo.name}: {prediccion}")
    else:
        print(f"{nodo.name}")
        for valor, probabilidad in prediccion.parameters[0].items(): 
            print(f"       {valor}: {probabilidad:.2f}")

lluvia
       ninguna: 0.77
       suave: 0.17
       fuerte: 0.06
mantenimiento
       no: 0.68
       si: 0.32
bus: a tiempo
cita
       atendida: 0.90
       perdida: 0.10
